# IBM Data Science Professional Certificate - Applied Data Science Capstone

# Introduction

In this notebook I will conduct an analysis on the accident severity data as a Applied Data Science Capstone Project of the IBM Data Science Professional Certificate programme.

The aim of this project is to predict the severity of an accident.


# Importing libraries and packages

In [1]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

# Download the dataset

In [2]:
!wget -O Data-Collisions.csv https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv

--2020-09-10 17:34:40--  https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
Resolving s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73917638 (70M) [text/csv]
Saving to: ‘Data-Collisions.csv’

100%[======================================>] 73,917,638  35.1MB/s   in 2.0s   

2020-09-10 17:34:42 (35.1 MB/s) - ‘Data-Collisions.csv’ saved [73917638/73917638]



# Having a high level look at the data

In [3]:
# read the data into a pandas dataframe and have a look at the first 10 rows

df = pd.read_csv('Data-Collisions.csv')

pd.set_option('display.max_columns', None) # option for showing all the columns when head or describe methods are used
df.head(5)

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE.1,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,5TH AVE NE AND NE 103RD ST,,NaN,2,Injury Collision,Angles,2,0,0,2,2013/03/27 00:00:00+00,3/27/2013 2:54:00 PM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Overcast,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N,NaN,NaN,1,Property Damage Only Collision,Sideswipe,2,0,0,2,2006/12/20 00:00:00+00,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),16,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",NaN,0,Raining,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST,NaN,NaN,1,Property Damage Only Collision,Parked Car,4,0,0,3,2004/11/18 00:00:00+00,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),14,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,0,Overcast,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,2ND AVE BETWEEN MARION ST AND MADISON ST,,NaN,1,Property Damage Only Collision,Other,3,0,0,3,2013/03/29 00:00:00+00,3/29/2013 9:26:00 AM,Mid-Block (not related to intersection),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,SWIFT AVE S AND SWIFT AV OFF RP,NaN,NaN,2,Injury Collision,Angles,2,0,0,2,2004/01/28 00:00:00+00,1/28/2004 8:04:00 AM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,0,Raining,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [4]:
# check the dimensions of the dataset

df.shape

(194673, 38)

In [5]:
# check the data types of all attributes in the dataset

df.dtypes

SEVERITYCODE        int64
X                 float64
Y                 float64
OBJECTID            int64
INCKEY              int64
COLDETKEY           int64
REPORTNO           object
STATUS             object
ADDRTYPE           object
INTKEY            float64
LOCATION           object
EXCEPTRSNCODE      object
EXCEPTRSNDESC      object
SEVERITYCODE.1      int64
SEVERITYDESC       object
COLLISIONTYPE      object
PERSONCOUNT         int64
PEDCOUNT            int64
PEDCYLCOUNT         int64
VEHCOUNT            int64
INCDATE            object
INCDTTM            object
JUNCTIONTYPE       object
SDOT_COLCODE        int64
SDOT_COLDESC       object
INATTENTIONIND     object
UNDERINFL          object
WEATHER            object
ROADCOND           object
LIGHTCOND          object
PEDROWNOTGRNT      object
SDOTCOLNUM        float64
SPEEDING           object
ST_COLCODE         object
ST_COLDESC         object
SEGLANEKEY          int64
CROSSWALKKEY        int64
HITPARKEDCAR       object
dtype: objec

In [6]:
# check the statistical summary of all integer or float64 type attributes in the dataset

df.describe()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,INTKEY,SEVERITYCODE.1,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,SDOT_COLCODE,SDOTCOLNUM,SEGLANEKEY,CROSSWALKKEY
count,194673.000000,189339.000000,189339.000000,194673.000000,194673.000000,194673.000000,65070.000000,194673.000000,194673.000000,194673.000000,194673.000000,194673.000000,194673.000000,1.149360e+05,194673.000000,1.946730e+05
mean,1.298901,-122.330518,47.619543,108479.364930,141091.456350,141298.811381,37558.450576,1.298901,2.444427,0.037139,0.028391,1.920780,13.867768,7.972521e+06,269.401114,9.782452e+03
std,0.457778,0.029976,0.056157,62649.722558,86634.402737,86986.542110,51745.990273,0.457778,1.345929,0.198150,0.167413,0.631047,6.868755,2.553533e+06,3315.776055,7.226926e+04
min,1.000000,-122.419091,47.495573,1.000000,1001.000000,1001.000000,23807.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.007024e+06,0.000000,0.000000e+00
25%,1.000000,-122.348673,47.575956,54267.000000,70383.000000,70383.000000,28667.000000,1.000000,2.000000,0.000000,0.000000,2.000000,11.000000,6.040015e+06,0.000000,0.000000e+00
50%,1.000000,-122.330224,47.615369,106912.000000,123363.000000,123363.000000,29973.000000,1.000000,2.000000,0.000000,0.000000,2.000000,13.000000,8.023022e+06,0.000000,0.000000e+00
75%,2.000000,-122.311937,47.663664,162272.000000,203319.000000,203459.000000,33973.000000,2.000000,3.000000,0.000000,0.000000,2.000000,14.000000,1.015501e+07,0.000000,0.000000e+00
max,2.000000,-122.238949,47.734142,219547.000000,331454.000000,332954.000000,757580.000000,2.000000,81.000000,6.000000,2.000000,12.000000,69.000000,1.307202e+07,525241.000000,5.239700e+06


In [7]:
# check the statistical summary of all types of attributes in the dataset

df.describe(include = "all")

# Execute the code below to see the statistical summary of a desired attribute
# df[['column1','column2']].describe()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE.1,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
count,194673.000000,189339.000000,189339.000000,194673.000000,194673.000000,194673.000000,194673,194673,192747,65070.000000,191996,84811,5638,194673.000000,194673,189769,194673.000000,194673.000000,194673.000000,194673.000000,194673,194673,188344,194673.000000,194673,29805,189789,189592,189661,189503,4667,1.149360e+05,9333,194655,189769,194673.000000,1.946730e+05,194673
unique,NaN,NaN,NaN,NaN,NaN,NaN,194670,2,3,NaN,24102,2,1,NaN,2,10,NaN,NaN,NaN,NaN,5985,162058,7,NaN,39,1,4,11,9,9,1,NaN,1,115,62,NaN,NaN,2
top,NaN,NaN,NaN,NaN,NaN,NaN,1776526,Matched,Block,NaN,BATTERY ST TUNNEL NB BETWEEN ALASKAN WY VI NB ...,,"Not Enough Information, or Insufficient Locati...",NaN,Property Damage Only Collision,Parked Car,NaN,NaN,NaN,NaN,2006/11/02 00:00:00+00,11/2/2006,Mid-Block (not related to intersection),NaN,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",Y,N,Clear,Dry,Daylight,Y,NaN,Y,32,One parked--one moving,NaN,NaN,N
freq,NaN,NaN,NaN,NaN,NaN,NaN,2,189786,126926,NaN,276,79173,5638,NaN,136485,47987,NaN,NaN,NaN,NaN,96,96,89800,NaN,85209,29805,100274,111135,124510,116137,4667,NaN,9333,27612,44421,NaN,NaN,187457
mean,1.298901,-122.330518,47.619543,108479.364930,141091.456350,141298.811381,NaN,NaN,NaN,37558.450576,NaN,NaN,NaN,1.298901,NaN,NaN,2.444427,0.037139,0.028391,1.920780,NaN,NaN,NaN,13.867768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.972521e+06,NaN,NaN,NaN,269.401114,9.782452e+03,NaN
std,0.457778,0.029976,0.056157,62649.722558,86634.402737,86986.542110,NaN,NaN,NaN,51745.990273,NaN,NaN,NaN,0.457778,NaN,NaN,1.345929,0.198150,0.167413,0.631047,NaN,NaN,NaN,6.868755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.553533e+06,NaN,NaN,NaN,3315.776055,7.226926e+04,NaN
min,1.000000,-122.419091,47.495573,1.000000,1001.000000,1001.000000,NaN,NaN,NaN,23807.000000,NaN,NaN,NaN,1.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.007024e+06,NaN,NaN,NaN,0.000000,0.000000e+00,NaN
25%,1.000000,-122.348673,47.575956,54267.000000,70383.000000,70383.000000,NaN,NaN,NaN,28667.000000,NaN,NaN,NaN,1.000000,NaN,NaN,2.000000,0.000000,0.000000,2.000000,NaN,NaN,NaN,11.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.040015e+06,NaN,NaN,NaN,0.000000,0.000000e+00,NaN
50%,1.000000,-122.330224,47.615369,106912.000000,123363.000000,123363.000000,NaN,NaN,NaN,29973.000000,NaN,NaN,NaN,1.000000,NaN,NaN,2.000000,0.000000,0.000000,2.000000,NaN,NaN,NaN,13.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.023022e+06,NaN,NaN,NaN,0.000000,0.000000e+00,NaN
75%,2.000000,-122.311937,47.663664,162272.000000,203319.000000,203459.000000,NaN,NaN,NaN,33973.000000,NaN,NaN,NaN,2.000000,NaN,NaN,3.000000,0.000000,0.000000,2.000000,NaN,NaN,NaN,14.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.015501e+07,NaN,NaN,NaN,0.000000,0.000000e+00,NaN


In [8]:
# look at the info of the dataset

df.info

<bound method DataFrame.info of         SEVERITYCODE           X          Y  OBJECTID  INCKEY  COLDETKEY  \
0                  2 -122.323148  47.703140         1    1307       1307   
1                  1 -122.347294  47.647172         2   52200      52200   
2                  1 -122.334540  47.607871         3   26700      26700   
3                  1 -122.334803  47.604803         4    1144       1144   
4                  2 -122.306426  47.545739         5   17700      17700   
5                  1 -122.387598  47.690575         6  320840     322340   
6                  1 -122.338485  47.618534         7   83300      83300   
7                  2 -122.320780  47.614076         9  330897     332397   
8                  1 -122.335930  47.611904        10   63400      63400   
9                  2 -122.384700  47.528475        12   58600      58600   
10                 1         NaN        NaN        14   48900      48900   
11                 1 -122.333831  47.547371        15   

In [9]:
# what is the distribution in the label column: SEVERITYCODE

df['SEVERITYCODE'].value_counts()

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

According to the attribute information in the dataset metadata, it looks like

136485 of all accidents lead to injury (1)

58188 of them lead to property damage (2)

# Data wrangling

In [10]:
# dealing with the missing data

missing_data = df.isnull()
missing_data.head(5)

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE.1,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,True,True,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,True,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,True,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,True,True,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False


In [11]:
# count missing values in each column

for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")

SEVERITYCODE
False    194673
Name: SEVERITYCODE, dtype: int64

X
False    189339
True       5334
Name: X, dtype: int64

Y
False    189339
True       5334
Name: Y, dtype: int64

OBJECTID
False    194673
Name: OBJECTID, dtype: int64

INCKEY
False    194673
Name: INCKEY, dtype: int64

COLDETKEY
False    194673
Name: COLDETKEY, dtype: int64

REPORTNO
False    194673
Name: REPORTNO, dtype: int64

STATUS
False    194673
Name: STATUS, dtype: int64

ADDRTYPE
False    192747
True       1926
Name: ADDRTYPE, dtype: int64

INTKEY
True     129603
False     65070
Name: INTKEY, dtype: int64

LOCATION
False    191996
True       2677
Name: LOCATION, dtype: int64

EXCEPTRSNCODE
True     109862
False     84811
Name: EXCEPTRSNCODE, dtype: int64

EXCEPTRSNDESC
True     189035
False      5638
Name: EXCEPTRSNDESC, dtype: int64

SEVERITYCODE.1
False    194673
Name: SEVERITYCODE.1, dtype: int64

SEVERITYDESC
False    194673
Name: SEVERITYDESC, dtype: int64

COLLISIONTYPE
False    189769
True       4904
Name: C

# Decide which columns will be removed before continuing

After investigating the attribute information in the dataset metadata and looking at the information in the cells (by head method) we can say that the following columns will not serve our goal will not be used in the feature set of our machine learning model.



In [35]:
# drop unnecessary columns and read the cleaned dataframe into a new dataframe

df_c = df.drop(['OBJECTID','INCKEY','COLDETKEY','REPORTNO','INTKEY','LOCATION','EXCEPTRSNCODE','STATUS','INATTENTIONIND','PEDROWNOTGRNT',
                      'EXCEPTRSNDESC','SEVERITYCODE.1','SEVERITYDESC','COLLISIONTYPE','JUNCTIONTYPE','SPEEDING',
                      'SDOT_COLCODE','SDOT_COLDESC','SDOTCOLNUM','ST_COLDESC','SEGLANEKEY',
                      'CROSSWALKKEY'], axis=1)

df_c.head(5)

,SEVERITYCODE,X,Y,ADDRTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,ST_COLCODE,HITPARKEDCAR
0,2,-122.323148,47.703140,Intersection,2,0,0,2,2013/03/27 00:00:00+00,3/27/2013 2:54:00 PM,N,Overcast,Wet,Daylight,10,N
1,1,-122.347294,47.647172,Block,2,0,0,2,2006/12/20 00:00:00+00,12/20/2006 6:55:00 PM,0,Raining,Wet,Dark - Street Lights On,11,N
2,1,-122.334540,47.607871,Block,4,0,0,3,2004/11/18 00:00:00+00,11/18/2004 10:20:00 AM,0,Overcast,Dry,Daylight,32,N
3,1,-122.334803,47.604803,Block,3,0,0,3,2013/03/29 00:00:00+00,3/29/2013 9:26:00 AM,N,Clear,Dry,Daylight,23,N
4,2,-122.306426,47.545739,Intersection,2,0,0,2,2004/01/28 00:00:00+00,1/28/2004 8:04:00 AM,0,Raining,Wet,Daylight,10,N


In [36]:
df_c.describe()

,SEVERITYCODE,X,Y,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT
count,194673.000000,189339.000000,189339.000000,194673.000000,194673.000000,194673.000000,194673.000000
mean,1.298901,-122.330518,47.619543,2.444427,0.037139,0.028391,1.920780
std,0.457778,0.029976,0.056157,1.345929,0.198150,0.167413,0.631047
min,1.000000,-122.419091,47.495573,0.000000,0.000000,0.000000,0.000000
25%,1.000000,-122.348673,47.575956,2.000000,0.000000,0.000000,2.000000
50%,1.000000,-122.330224,47.615369,2.000000,0.000000,0.000000,2.000000
75%,2.000000,-122.311937,47.663664,3.000000,0.000000,0.000000,2.000000
max,2.000000,-122.238949,47.734142,81.000000,6.000000,2.000000,12.000000


In [37]:
df_c.describe(include = "all")

,SEVERITYCODE,X,Y,ADDRTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,ST_COLCODE,HITPARKEDCAR
count,194673.000000,189339.000000,189339.000000,192747,194673.000000,194673.000000,194673.000000,194673.000000,194673,194673,189789,189592,189661,189503,194655,194673
unique,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,5985,162058,4,11,9,9,115,2
top,NaN,NaN,NaN,Block,NaN,NaN,NaN,NaN,2006/11/02 00:00:00+00,11/2/2006,N,Clear,Dry,Daylight,32,N
freq,NaN,NaN,NaN,126926,NaN,NaN,NaN,NaN,96,96,100274,111135,124510,116137,27612,187457
mean,1.298901,-122.330518,47.619543,NaN,2.444427,0.037139,0.028391,1.920780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.457778,0.029976,0.056157,NaN,1.345929,0.198150,0.167413,0.631047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,-122.419091,47.495573,NaN,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.000000,-122.348673,47.575956,NaN,2.000000,0.000000,0.000000,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.000000,-122.330224,47.615369,NaN,2.000000,0.000000,0.000000,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2.000000,-122.311937,47.663664,NaN,3.000000,0.000000,0.000000,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
df_c.shape

(194673, 16)

In [39]:
df_c.dtypes

SEVERITYCODE      int64
X               float64
Y               float64
ADDRTYPE         object
PERSONCOUNT       int64
PEDCOUNT          int64
PEDCYLCOUNT       int64
VEHCOUNT          int64
INCDATE          object
INCDTTM          object
UNDERINFL        object
WEATHER          object
ROADCOND         object
LIGHTCOND        object
ST_COLCODE       object
HITPARKEDCAR     object
dtype: object

# Check the missing data amount in the remaining attributes

In [40]:
missing_data_c = df_c.isnull()

for column in missing_data_c.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")

SEVERITYCODE
False    194673
Name: SEVERITYCODE, dtype: int64

X
False    189339
True       5334
Name: X, dtype: int64

Y
False    189339
True       5334
Name: Y, dtype: int64

ADDRTYPE
False    192747
True       1926
Name: ADDRTYPE, dtype: int64

PERSONCOUNT
False    194673
Name: PERSONCOUNT, dtype: int64

PEDCOUNT
False    194673
Name: PEDCOUNT, dtype: int64

PEDCYLCOUNT
False    194673
Name: PEDCYLCOUNT, dtype: int64

VEHCOUNT
False    194673
Name: VEHCOUNT, dtype: int64

INCDATE
False    194673
Name: INCDATE, dtype: int64

INCDTTM
False    194673
Name: INCDTTM, dtype: int64

UNDERINFL
False    189789
True       4884
Name: UNDERINFL, dtype: int64

WEATHER
False    189592
True       5081
Name: WEATHER, dtype: int64

ROADCOND
False    189661
True       5012
Name: ROADCOND, dtype: int64

LIGHTCOND
False    189503
True       5170
Name: LIGHTCOND, dtype: int64

ST_COLCODE
False    194655
True         18
Name: ST_COLCODE, dtype: int64

HITPARKEDCAR
False    194673
Name: HITPARKEDCAR, dtyp

# Drop all rows that do not have data in the selected attributes

In [41]:
df_c.dropna(subset=["X"], axis=0, inplace=True)
df_c.dropna(subset=["Y"], axis=0, inplace=True)
df_c.dropna(subset=["ADDRTYPE"], axis=0, inplace=True)
df_c.dropna(subset=["UNDERINFL"], axis=0, inplace=True)
df_c.dropna(subset=["WEATHER"], axis=0, inplace=True)
df_c.dropna(subset=["ROADCOND"], axis=0, inplace=True)
df_c.dropna(subset=["LIGHTCOND"], axis=0, inplace=True)

df_c.reset_index(drop=True, inplace=True)
df_c.shape

(184167, 16)

All the rows including missing data are removed from the dataset

In [42]:
df_c.head(5)

,SEVERITYCODE,X,Y,ADDRTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,ST_COLCODE,HITPARKEDCAR
0,2,-122.323148,47.703140,Intersection,2,0,0,2,2013/03/27 00:00:00+00,3/27/2013 2:54:00 PM,N,Overcast,Wet,Daylight,10,N
1,1,-122.347294,47.647172,Block,2,0,0,2,2006/12/20 00:00:00+00,12/20/2006 6:55:00 PM,0,Raining,Wet,Dark - Street Lights On,11,N
2,1,-122.334540,47.607871,Block,4,0,0,3,2004/11/18 00:00:00+00,11/18/2004 10:20:00 AM,0,Overcast,Dry,Daylight,32,N
3,1,-122.334803,47.604803,Block,3,0,0,3,2013/03/29 00:00:00+00,3/29/2013 9:26:00 AM,N,Clear,Dry,Daylight,23,N
4,2,-122.306426,47.545739,Intersection,2,0,0,2,2004/01/28 00:00:00+00,1/28/2004 8:04:00 AM,0,Raining,Wet,Daylight,10,N


In [43]:
df_c.describe()

,SEVERITYCODE,X,Y,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT
count,184167.000000,184167.000000,184167.000000,184167.000000,184167.000000,184167.000000,184167.000000
mean,1.304142,-122.330502,47.619544,2.468048,0.038845,0.029745,1.973692
std,0.460045,0.030018,0.056215,1.358602,0.202556,0.171220,0.556182
min,1.000000,-122.419091,47.495573,0.000000,0.000000,0.000000,0.000000
25%,1.000000,-122.348846,47.575884,2.000000,0.000000,0.000000,2.000000
50%,1.000000,-122.330168,47.615361,2.000000,0.000000,0.000000,2.000000
75%,2.000000,-122.311922,47.663835,3.000000,0.000000,0.000000,2.000000
max,2.000000,-122.238949,47.734142,81.000000,6.000000,2.000000,12.000000


# Investigate the values in the attributes that are more likely to be features for our model

In [44]:
df_c['UNDERINFL'].value_counts()

N    97244
0    78064
Y     4980
1     3879
Name: UNDERINFL, dtype: int64

Since there are two type of binary categorization in the UNDERINFL attribute, we should change it to single type.

In [45]:
df_c.replace("N","0", inplace = True)
df_c.replace("Y","1", inplace = True)

df_c['UNDERINFL'].value_counts()

0    175308
1      8859
Name: UNDERINFL, dtype: int64

In [46]:
df_c['WEATHER'].value_counts()

Clear                       108833
Raining                      31987
Overcast                     27105
Unknown                      13846
Snowing                        888
Other                          765
Fog/Smog/Smoke                 553
Sleet/Hail/Freezing Rain       112
Blowing Sand/Dirt               49
Severe Crosswind                24
Partly Cloudy                    5
Name: WEATHER, dtype: int64

In [47]:
df_c['ROADCOND'].value_counts()

Dry               121871
Wet                46009
Unknown            13795
Ice                 1174
Snow/Slush           984
Other                116
Standing Water       102
Sand/Mud/Dirt         63
Oil                   53
Name: ROADCOND, dtype: int64

In [48]:
df_c['LIGHTCOND'].value_counts()

Daylight                    113522
Dark - Street Lights On      47250
Unknown                      12416
Dusk                          5763
Dawn                          2422
Dark - No Street Lights       1450
Dark - Street Lights Off      1145
Other                          188
Dark - Unknown Lighting         11
Name: LIGHTCOND, dtype: int64

In [49]:
df_c['HITPARKEDCAR'].value_counts()

0    177537
1      6630
Name: HITPARKEDCAR, dtype: int64

# Change the attribute types as required
Numerical variables should have type 'float' or 'int', and variables with strings such as categories should have type 'object'. 

In [50]:
# change the date and time attributes to date format

df_c['INCDATE'] = pd.to_datetime(df_c['INCDATE'])
df_c['INCDTTM'] = pd.to_datetime(df_c['INCDTTM'])
df_c.head(5)

,SEVERITYCODE,X,Y,ADDRTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,ST_COLCODE,HITPARKEDCAR
0,2,-122.323148,47.703140,Intersection,2,0,0,2,2013-03-27 00:00:00+00:00,2013-03-27 14:54:00,0,Overcast,Wet,Daylight,10,0
1,1,-122.347294,47.647172,Block,2,0,0,2,2006-12-20 00:00:00+00:00,2006-12-20 18:55:00,0,Raining,Wet,Dark - Street Lights On,11,0
2,1,-122.334540,47.607871,Block,4,0,0,3,2004-11-18 00:00:00+00:00,2004-11-18 10:20:00,0,Overcast,Dry,Daylight,32,0
3,1,-122.334803,47.604803,Block,3,0,0,3,2013-03-29 00:00:00+00:00,2013-03-29 09:26:00,0,Clear,Dry,Daylight,23,0
4,2,-122.306426,47.545739,Intersection,2,0,0,2,2004-01-28 00:00:00+00:00,2004-01-28 08:04:00,0,Raining,Wet,Daylight,10,0


In [51]:
df_c.dtypes

SEVERITYCODE                  int64
X                           float64
Y                           float64
ADDRTYPE                     object
PERSONCOUNT                   int64
PEDCOUNT                      int64
PEDCYLCOUNT                   int64
VEHCOUNT                      int64
INCDATE         datetime64[ns, UTC]
INCDTTM              datetime64[ns]
UNDERINFL                    object
WEATHER                      object
ROADCOND                     object
LIGHTCOND                    object
ST_COLCODE                   object
HITPARKEDCAR                 object
dtype: object

# Investigate the effect of attributes on the accident severity

In [52]:
df_c.groupby(['UNDERINFL'])['SEVERITYCODE'].value_counts(normalize=True)

UNDERINFL  SEVERITYCODE
0          1               0.700305
           2               0.299695
1          1               0.607856
           2               0.392144
Name: SEVERITYCODE, dtype: float64

In [53]:
df_c.groupby(['WEATHER'])['SEVERITYCODE'].value_counts(normalize=True)

WEATHER                   SEVERITYCODE
Blowing Sand/Dirt         1               0.734694
                          2               0.265306
Clear                     1               0.675926
                          2               0.324074
Fog/Smog/Smoke            1               0.667269
                          2               0.332731
Other                     1               0.856209
                          2               0.143791
Overcast                  1               0.682605
                          2               0.317395
Partly Cloudy             2               0.600000
                          1               0.400000
Raining                   1               0.660612
                          2               0.339388
Severe Crosswind          1               0.708333
                          2               0.291667
Sleet/Hail/Freezing Rain  1               0.758929
                          2               0.241071
Snowing                   1               0

In [54]:
df_c.groupby(['ROADCOND'])['SEVERITYCODE'].value_counts(normalize=True)

ROADCOND        SEVERITYCODE
Dry             1               0.676699
                2               0.323301
Ice             1               0.773424
                2               0.226576
Oil             1               0.622642
                2               0.377358
Other           1               0.663793
                2               0.336207
Sand/Mud/Dirt   1               0.650794
                2               0.349206
Snow/Slush      1               0.832317
                2               0.167683
Standing Water  1               0.745098
                2               0.254902
Unknown         1               0.948387
                2               0.051613
Wet             1               0.666109
                2               0.333891
Name: SEVERITYCODE, dtype: float64

In [55]:
df_c.groupby(['LIGHTCOND'])['SEVERITYCODE'].value_counts(normalize=True)

LIGHTCOND                 SEVERITYCODE
Dark - No Street Lights   1               0.780000
                          2               0.220000
Dark - Street Lights Off  1               0.732751
                          2               0.267249
Dark - Street Lights On   1               0.700127
                          2               0.299873
Dark - Unknown Lighting   1               0.636364
                          2               0.363636
Dawn                      1               0.665566
                          2               0.334434
Daylight                  1               0.666373
                          2               0.333627
Dusk                      1               0.668402
                          2               0.331598
Other                     1               0.803191
                          2               0.196809
Unknown                   1               0.953044
                          2               0.046956
Name: SEVERITYCODE, dtype: float64

# Data visualization

# Modeling with UNDERINFL attribution

# Modeling with WEATHER attribution

# Modeling with ROADCOND attribution

# Modeling with LIGHTCOND attribution

# Modeling with all attributions